In [38]:
import pandas as pd
import nltk
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import re
import gensim
from gensim.models import KeyedVectors
import numpy as np
from numpy import dot
from numpy.linalg import norm
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases
from nltk import sent_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
import gensim.downloader as api

In [110]:
custom_stopwords = ['echo', 'alexa', 'dot', "star", 'amazon', 'prime', '2nd', 'generation', "fire", "stick", "firestick", "skype", "facetime", '1st', '3rd', '4th', '5th', "hub", "hulu", 'google', 'netflix', 'youtube', 'philip', 'tp-link', 'fourth', 'roku', "siri", 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'nor', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "...", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', 'didn', 'doesn', 'hadn', 'hasn', 'haven', 'isn', 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', 'wasn', 'weren', 'won', 'wouldn']

def negation_handler(sentence):	
    temp = int(0)
    for i in range(len(sentence)):
        if sentence[i-1] in ['not',"n't", "no", "without"]:
            antonyms = []
            for syn in wordnet.synsets(sentence[i]):
                syns = wordnet.synsets(sentence[i])
                w1 = syns[0].name()
                temp = 0
                for l in syn.lemmas():
                    if l.antonyms():
                        antonyms.append(l.antonyms()[0].name())
                max_dissimilarity = 0
                for ant in antonyms:
                    syns = wordnet.synsets(ant)
                    w2 = syns[0].name()
                    syns = wordnet.synsets(sentence[i])
                    w1 = syns[0].name()
                    word1 = wordnet.synset(w1)
                    word2 = wordnet.synset(w2)
                    if isinstance(word1.wup_similarity(word2), float) or isinstance(word1.wup_similarity(word2), int):
                        temp = 1 - word1.wup_similarity(word2)
                    if temp>max_dissimilarity:
                        max_dissimilarity = temp
                        antonym_max = ant
                        sentence[i] = antonym_max
                        sentence[i-1] = ''
    while '' in sentence:
        sentence.remove('')
    return sentence

In [111]:
dataset = pd.read_csv("amazon_alexa.tsv", sep = "\t")
dataset.drop(dataset[dataset.rating == 3].index, inplace=True) #droppa recensioni con 3-4 stelle
dataset.drop(dataset[dataset.rating == 4].index, inplace=True) #droppa recensioni con 3-4 stelle
dataset.tail()

,rating,date,variation,verified_reviews,feedback
3144,5,30-Jul-18,Black Dot,love it,1
3145,5,30-Jul-18,Black Dot,"Perfect for kids, adults and everyone in betwe...",1
3146,5,30-Jul-18,Black Dot,"Listening to music, searching locations, check...",1
3147,5,30-Jul-18,Black Dot,"I do love these things, i have them running my...",1
3148,5,30-Jul-18,White Dot,Only complaint I have is that the sound qualit...,1


In [112]:
#  CREAZIONE DI UN SAMPLE DATASET BILANCIATO
# prende le prime n recensioni positive di lunghezza maggiore, dove n è il numero di recensioni negative
def create_balanced_dataset(dataset):
    """Bilancia il dataset uniformando il numero di recensioni negative e positive. Prende in input il dataset"""
    reviews_1 = list(dataset[dataset["feedback"] == 1]["verified_reviews"])
    reviews_0 = list(dataset[dataset["feedback"] == 0]["verified_reviews"])
    reviews_1.sort(key=len, reverse = True)
    sample_1 = reviews_1[:len(reviews_0)]
    verified_reviews_sample = []
    feedback_sample = []
    verified_reviews_sample.extend(sample_1)
    verified_reviews_sample.extend(reviews_0)
    feedback_sample.extend([1 for i in range(len(sample_1))])
    feedback_sample.extend([0 for i in range(len(reviews_0))])
    dataset = pd.DataFrame({"verified_reviews":verified_reviews_sample, "feedback": feedback_sample})
    print(dataset["feedback"].value_counts())
    return dataset


# dataset = create_balanced_dataset(dataset)

In [113]:
def get_wordnet_pos(treebank_tag):
    """
    return WORDNET POS compliance to WORDENT lemmatization (a,n,r,v) 
    """
    if treebank_tag.startswith('J'):
        return "a"
    elif treebank_tag.startswith('V'):
        return "v"
    elif treebank_tag.startswith('N'):
        return "n"
    elif treebank_tag.startswith('R'):
        return "r"
    else:
        return "n"
        
pos_list = ["JJ", "JJR", "JJS", "RB", "RBR", "RBS", "VB", "VBD", "VBG", "VBN", "VBP", "VBZ"]

def tokenize_list_of_text(list_of_text, custom_stopwords = [], pos_filter = False, pos_list = []):
    """Tokenizza tutte le recensioni, pulisce da stopwords, elimina token <= 2 caratteri e lemmatizza.
    Ritorna sia il la lista tokenizzata ma come stringa sia come lista di tokens, dunque ritorna due elementi"""

    tokenizer = nltk.tokenize.TweetTokenizer()
    lemmatizer = nltk.WordNetLemmatizer()
    detokenizer = TreebankWordDetokenizer()

    tokenized_reviews = []
    sent_tokenized_reviews = []
    pos_reviews = []
    for review in list_of_text: #pulisce le recensioni
        review = re.sub(r'\d+', '', review)
        clean_text = ""
        tokens = nltk.tokenize.word_tokenize(review, language='english', preserve_line=False)
        tokens = negation_handler(tokens)
        tokens = [w.lower() for w in tokens]
        tokens_pos = pos_tag(tokens)
        lemmatized_tokens = [(lemmatizer.lemmatize(w, get_wordnet_pos(pos)), pos) for w, pos in tokens_pos]
        if pos_filter:
            clean_tokens = [(w, pos) for w, pos in lemmatized_tokens if w not in string.punctuation and len(w)>2 and w not in custom_stopwords and pos in pos_list]
        else:
            clean_tokens = [(w, pos) for w, pos in lemmatized_tokens if w not in string.punctuation and len(w)>2 and w not in custom_stopwords]
        sent_tokenized_reviews.append([w for w, pos in clean_tokens])
        tokenized_reviews.append(detokenizer.detokenize([w for w, pos in clean_tokens]))
    
    n_tokens = []
    for sent in sent_tokenized_reviews:
        for w in sent:
            n_tokens.append(w)
    print("total number of tokens extracted are:", len(set(n_tokens)))
    return tokenized_reviews,  sent_tokenized_reviews # ritorna una tupla!

In [114]:
new_text, new_sent_tok = tokenize_list_of_text(dataset["verified_reviews"], custom_stopwords, False, pos_list)
# contengono una lista di tutte le frasi pre processate, nella prima variabile in stringa, nella seconda in tokens 
bigrams = Phrases(new_sent_tok)
bigrams[new_sent_tok][12]

total number of tokens extracted are: 2766


['expect', '....']

In [115]:
tot_tokens = []

for sent in [x for x in bigrams[new_sent_tok]]:
    for tok in sent:
        tot_tokens.append(tok)

freqs = nltk.FreqDist(tot_tokens)
cleaned_reviews = []

for sent in [x for x in bigrams[new_sent_tok]]:
    clean_sent = []
    for tok in sent:
        if freqs[tok] > 4:
            clean_sent.append(tok)
    cleaned_reviews.append(clean_sent)

pos_reviews = [["$".join(x) for x in pos_tag(rev)] for rev in cleaned_reviews]

In [116]:
neg_tokenized = [rev for rev, feedback in zip(cleaned_reviews, dataset["feedback"]) if feedback == 0]
pos_tokenized = [rev for rev, feedback in zip(cleaned_reviews, dataset["feedback"]) if feedback == 1]
neg_tokenized = [w for rev in neg_tokenized for w in rev]
pos_tokenized = [w for rev in pos_tokenized for w in rev]

import math 

def get_pos_neg_score(pos, neg, threshold = 0.30):
    total_tokens = pos + neg
    total_tokens_freq = nltk.FreqDist(total_tokens)
    pos_freq = nltk.FreqDist(pos)
    neg_freq = nltk.FreqDist(neg)
    data_dict = {"pos": {}, "neg": {}}

    for w in pos:
        p = total_tokens_freq[w] / len(total_tokens)
        p_category = pos_freq[w] / total_tokens_freq[w]
        h = -math.log2(p)
        score = p_category
        data_dict["pos"][w] = score
    
    for w in neg:
        p = total_tokens_freq[w] / len(total_tokens)
        p_category = neg_freq[w] / total_tokens_freq[w]
        h = -math.log2(p)
        score = p_category
        data_dict["neg"][w] = score

    risultato = {}

    for label in ["pos", "neg"]:
        values_list = list(data_dict[label].values())
        min_values = min(values_list)
        max_values = max(values_list)
        for k in data_dict[label]:
            val = data_dict[label][k]
            data_dict[label][k] = (val- min_values)/(max_values-min_values)

    for w in total_tokens:

        try:
            pos_score = data_dict["pos"][w]
        except KeyError:
            risultato[w] = 1
            continue

        try:
            neg_score = data_dict["neg"][w]
        except KeyError:
            risultato[w] = 1
            continue


        risultato[w] = max([pos_score, neg_score])

    # return data_dict
    return {k: v for k, v in sorted(risultato.items(), key = lambda item: item[1], reverse=True)}
    
data_dict = get_pos_neg_score(pos_tokenized, neg_tokenized, threshold=0)
data_dict

{'lot_fun': 1,
 'control_light': 1,
 'play_game': 1,
 'every_room': 1,
 'throughout_house': 1,
 'dad': 1,
 'laugh': 1,
 'mother': 1,
 'enjoy': 1,
 'cook': 1,
 'happy_purchase': 1,
 'easy_set': 1,
 'toy': 1,
 'improvement': 1,
 'specific': 1,
 'flash_briefing': 1,
 'every_morning': 1,
 'custom': 1,
 "'re": 1,
 'amazing': 1,
 'advertised': 1,
 'mainly': 1,
 'favorite': 1,
 'tune': 1,
 'still_learn': 1,
 'capability': 1,
 'learning': 1,
 'daughter': 1,
 'base': 1,
 'include': 1,
 'place': 1,
 'best': 1,
 'husband': 1,
 'satisfied': 1,
 'excelente': 1,
 'que': 1,
 'sale': 1,
 'phenomenal': 1,
 'easy_setup': 1,
 'others': 1,
 'tell_joke': 1,
 'favorite_song': 1,
 'available': 1,
 'primarily': 1,
 'thanks': 1,
 'excellent': 1,
 'tech_savvy': 1,
 'wonderful': 1,
 'father_day': 1,
 'forget': 1,
 'type': 1,
 'additional': 1,
 'traffic': 1,
 'trivia': 1,
 'entire': 1,
 'homework': 1,
 'grandchild': 1,
 'bonus': 1,
 'exactly': 1,
 'classroom': 1,
 'gadget': 1,
 'anymore': 1,
 'outside': 1,
 'gran

In [117]:
w2v_model = Word2Vec(cleaned_reviews, vector_size=300, window = 5, min_count = 0, sg=1, hs = 1, alpha=0.03, min_alpha=0.0007)

In [118]:
w2v_model.train(cleaned_reviews, total_examples=len(cleaned_reviews), epochs=100, report_delay=1)

(1664176, 2193800)

In [119]:
w2v_model.wv.most_similar("great", topn = 10)

[('work', 0.2414400726556778),
 ('love', 0.20556379854679108),
 ('speed', 0.18921929597854614),
 ('charm', 0.18912632763385773),
 ('shipping', 0.17311447858810425),
 ('satisfied', 0.16941535472869873),
 ('product', 0.1688525229692459),
 ('understands', 0.16731740534305573),
 ('packaging', 0.1669285148382187),
 ('outstanding', 0.16080991923809052)]

In [120]:
w2v_model.wv.most_similar("love", topn = 2000)[-10:]

[('surrender', -0.028355229645967484),
 ('cycle', -0.02960037998855114),
 ('issue', -0.03215289115905762),
 ('equipment', -0.0376964770257473),
 ('next', -0.038393303751945496),
 ('expensive', -0.038506392389535904),
 ('software', -0.0453251488506794),
 ('lock', -0.04733620956540108),
 ('app', -0.048398081213235855),
 ('actually', -0.053606465458869934)]

In [121]:
tfidf = TfidfVectorizer(tokenizer=lambda i:i, lowercase=False, min_df = 0)
tfidf_model = tfidf.fit_transform(pos_reviews)
print(tfidf_model[90,:].data)
# print(pos_reviews[90])
# for feat,weight in zip(tfidf.inverse_transform(tfidf_model[10,:])[0],tfidf_model[10,:].data):
#     print(feat,round(weight,4))
tfidf.inverse_transform(tfidf_model[90,:])[0]

[0.4959246  0.58085256 0.51257842 0.25393915 0.29907752]


array(['happy$JJ', 'full$JJ', 'size$NN', 'great$JJ', 'work$NN'],
      dtype='<U20')

In [127]:
g_model = []
google_model = api.load('word2vec-google-news-300', True)
g_model = KeyedVectors.load_word2vec_format(google_model, binary=True)

In [123]:
def review_vectors(tokens, size, weights, pretrained = False):
    vec = np.zeros(size).reshape((1, size))
    count = 0
    if len(tokens) != len(weights):
        print("nope")
    for word, weight in zip(tokens, weights):
        word = word.split("$")[0]
        try:
            if pretrained:
                vec += g_model[word] * weight
            else:
                vec += w2v_model.wv[word] * weight
            count +=1
        except KeyError:
            print("non trovo", word)
            continue

    if count!= 0:
        # vec /= count
        vec = vec / norm(vec)
        
    return vec

In [131]:
w2v_reviews = np.zeros((len(cleaned_reviews), 300))
for i in range(len(cleaned_reviews)):
    w2v_reviews[i,:] = review_vectors(tfidf.inverse_transform(tfidf_model[i, :])[0], 300, tfidf_model[i,:].data, False)
w2v_df = pd.DataFrame(w2v_reviews)
w2v_df

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.051965,0.032598,0.092820,-0.090291,0.011296,-0.106433,-0.035136,0.087942,0.000669,-0.034708,...,0.031660,0.049640,0.089187,-0.108674,0.016770,-0.046675,0.017672,0.059694,-0.056233,0.049215
1,-0.051965,0.032598,0.092820,-0.090291,0.011296,-0.106433,-0.035136,0.087942,0.000669,-0.034708,...,0.031660,0.049640,0.089187,-0.108674,0.016770,-0.046675,0.017672,0.059694,-0.056233,0.049215
2,0.041048,0.075349,0.008408,0.044758,0.006526,-0.003864,0.126463,0.150340,-0.042830,-0.017194,...,-0.106423,0.009752,0.050289,-0.015290,0.039383,0.089870,0.013294,-0.041030,-0.033841,0.009543
3,-0.079484,-0.030426,0.010099,0.027317,0.118966,-0.095691,0.062336,0.069285,-0.057235,0.088395,...,-0.019002,-0.065988,0.021667,-0.014585,0.057510,-0.020091,0.000885,0.048733,0.096220,-0.030397
4,0.032622,0.038873,-0.021016,-0.035260,0.022223,-0.029787,0.086684,0.126250,-0.008218,-0.125110,...,0.000356,0.061455,0.047957,0.017249,0.138817,0.089834,-0.056131,0.017183,0.008048,-0.019058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2538,-0.051965,0.032598,0.092820,-0.090291,0.011296,-0.106433,-0.035136,0.087942,0.000669,-0.034708,...,0.031660,0.049640,0.089187,-0.108674,0.016770,-0.046675,0.017672,0.059694,-0.056233,0.049215
2539,0.015359,0.066549,-0.090542,0.072321,0.040601,-0.123594,0.081738,0.080564,-0.082635,0.006373,...,0.026246,-0.062868,-0.035900,-0.064638,-0.037727,0.067814,0.049687,-0.034761,0.046165,0.044742
2540,0.064834,0.025239,0.024579,0.052118,-0.003706,-0.013177,0.113357,0.166890,-0.056186,0.058405,...,-0.014681,0.030471,0.048655,-0.120014,0.016001,0.043002,0.057486,-0.051897,0.042263,0.004869
2541,-0.047042,0.097701,-0.015553,-0.017673,-0.000264,-0.111854,0.036752,0.204240,0.003434,-0.080003,...,-0.065981,-0.024913,0.010814,0.004995,0.011272,0.169864,0.019965,0.014383,0.058099,-0.022527


In [132]:
X_train, X_test, Y_train, Y_test = train_test_split(w2v_reviews, dataset["feedback"].values, test_size=0.2, random_state=10)
Y_train

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [133]:
svm = LinearSVC(max_iter=3000)
svm_model = svm.fit(X_train, Y_train)
predictions = svm_model.predict(X_test)
print(metrics.classification_report(Y_test, predictions))

              precision    recall  f1-score   support

           0       0.74      0.57      0.64        44
           1       0.96      0.98      0.97       465

    accuracy                           0.94       509
   macro avg       0.85      0.77      0.81       509
weighted avg       0.94      0.94      0.94       509



In [19]:
# risultati migliori:

# w2v + tf-idf weights considering pos (good$JJ) disambiguation also! + svm and word count > 4

# w2v configuration:
# w2v_model = Word2Vec(cleaned_reviews, vector_size=300, window = 5, min_count = 0, sg=1, hs = 1, alpha=0.03, min_alpha=0.0007) and 100 epochs for training

# otherwise the balanced dataset performs better
